In [1]:
import pandas as pd
import time

dataset=pd.read_csv("cleaned_data.csv",index_col=0)

#remove Nan Values from the table

dataset=dataset.dropna()

In [2]:
import numpy as np

np.random.seed(10)

random_number=np.random.rand(len(dataset.index.values))

msk1=random_number<0.7
msk2=random_number>0.85
msk3=(random_number>=0.7) & (random_number<=0.85)

train_set=dataset.loc[msk1,:]
test_set=dataset.loc[msk2,:]
validation_set=dataset.loc[msk3,:]

In [3]:
train_set.head()

,loan_amnt,term_digit,int_rate,home_own_binary,annual_inc,purpose_binary,earliest_cre_line,inq_last_6mths,binary_default
1,2500.0,6,15.27,1,30000.0,1,20179.0,5.0,0
2,2400.0,3,15.96,1,12252.0,2,21124.0,2.0,1
4,3000.0,6,12.69,1,80000.0,3,18993.0,0.0,1
5,5000.0,3,7.90,1,36000.0,4,22220.0,3.0,1
6,7000.0,6,15.96,1,47004.0,5,22462.0,1.0,1


In [4]:
import tensorflow as tf

#set placeholder for 
input_x=tf.placeholder(dtype=tf.float32,shape=(None,8))
input_y=tf.placeholder(dtype=tf.int32,shape=(None))

#set variables to calculate the number of 

n_neural=10

#initialize the transfer matrix and bias term
w1 = tf.Variable(tf.truncated_normal([8, n_neural], stddev=0.1), name="W")
b1=tf.Variable(tf.zeros([n_neural]), name="B",dtype=tf.float32)

w2 = tf.Variable(tf.truncated_normal([n_neural,2], stddev=0.1), name="W")
b2=tf.Variable(tf.zeros([2]), name="B",dtype=tf.float32)

#construct the hidden layer layer

net=tf.nn.relu(tf.add(tf.matmul(input_x,w1),b1))

#output layer

logits=tf.add(tf.matmul(net,w2),b2)

#construct the loss function
loss=tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits,labels=input_y))

#optimization fucntion
learning_rate = 0.001
optimizer = tf.train.AdamOptimizer(learning_rate)
training_op = optimizer.minimize(loss)

probability=tf.nn.softmax(logits)
classification=tf.nn.in_top_k(logits, input_y, 1)
accuracy=tf.reduce_mean(tf.cast(classification,tf.float32))

/home/ubuntu/miniconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [5]:
init = tf.global_variables_initializer()

batch_size=50
start_time=time.time()
with tf.Session() as sess:
    init.run()
    
    for iteration in range(len(train_set.index.values)//batch_size):
        X=train_set.iloc[(iteration*batch_size):((iteration+1)*batch_size),0:8]
        y=train_set.iloc[(iteration*batch_size):((iteration+1)*batch_size),8]
        lo=sess.run([training_op],feed_dict={input_x:X,input_y:y})
    
    train_accuracy=accuracy.eval(feed_dict={input_x:X,input_y:y})
    
    #running test set and validation set
    
    test_prediction,test_accuracy=sess.run([probability,accuracy],feed_dict={input_x:test_set.iloc[:,0:8],input_y:test_set.iloc[:,8]})
    print(list(zip(test_set.iloc[:,8],test_prediction))[0:10])
    print(test_accuracy)
    
    validate_prediction,validate_accuracy=sess.run([probability,accuracy],feed_dict={input_x:validation_set.iloc[:,0:8],input_y:validation_set.iloc[:,8]})
    #print(zip(validation_set.iloc[:,8],validate_prediction))
    print(validate_accuracy)
    sess.close()
    print("With No Scaling, the algorithm takes --- %s seconds ---" % (time.time() - start_time))

[(1, array([  2.10464805e-07,   9.99999762e-01], dtype=float32)), (1, array([  3.71838489e-08,   1.00000000e+00], dtype=float32)), (1, array([ 0.01338244,  0.98661751], dtype=float32)), (1, array([  1.98607381e-06,   9.99997973e-01], dtype=float32)), (1, array([ 0.01804949,  0.98195052], dtype=float32)), (0, array([  1.20440905e-04,   9.99879599e-01], dtype=float32)), (1, array([  1.17984435e-07,   9.99999881e-01], dtype=float32)), (1, array([  8.24799878e-04,   9.99175131e-01], dtype=float32)), (1, array([  1.83378463e-04,   9.99816597e-01], dtype=float32)), (1, array([  4.83868753e-05,   9.99951601e-01], dtype=float32))]
0.923228
0.923206
With No Scaling, the algorithm takes --- 18.77990961074829 seconds ---


In [6]:
train_set.head()

,loan_amnt,term_digit,int_rate,home_own_binary,annual_inc,purpose_binary,earliest_cre_line,inq_last_6mths,binary_default
1,2500.0,6,15.27,1,30000.0,1,20179.0,5.0,0
2,2400.0,3,15.96,1,12252.0,2,21124.0,2.0,1
4,3000.0,6,12.69,1,80000.0,3,18993.0,0.0,1
5,5000.0,3,7.90,1,36000.0,4,22220.0,3.0,1
6,7000.0,6,15.96,1,47004.0,5,22462.0,1.0,1


In [7]:
#design the function to resclae the data to adjust the loan amount, interest rate, annual income and earliest credit line
from sklearn import preprocessing

def scale_train_data(df):
    target=['loan_amnt','int_rate','annual_inc','earliest_cre_line']
    new_df=df.loc[:,df.columns.difference(target)]
    loan_amount_scaler=preprocessing.StandardScaler().fit(df[target[0]].reshape(-1, 1))
    new_df['scaled_'+target[0]]=loan_amount_scaler.transform(df[target[0]].reshape(-1, 1))
    
    int_rate_scaler=preprocessing.StandardScaler().fit(df[target[1]].reshape(-1, 1))
    new_df['scaled_'+target[1]]=int_rate_scaler.transform(df[target[1]].reshape(-1, 1))
    
    annual_inc_scaler=preprocessing.StandardScaler().fit(df[target[2]].reshape(-1, 1))
    new_df['scaled_'+target[2]]=annual_inc_scaler.transform(df[target[2]].reshape(-1, 1))
    
    earliest_cre_line_scaler=preprocessing.StandardScaler().fit(df[target[3]].reshape(-1, 1))
    new_df['scaled_'+target[3]]=earliest_cre_line_scaler.transform(df[target[3]].reshape(-1, 1))
    
    return new_df,[loan_amount_scaler,int_rate_scaler,annual_inc_scaler,earliest_cre_line_scaler]


In [8]:
#design the function to apply the scaler onto test and validation set
def apply_scaler(df,scaler_list):
    target=['loan_amnt','int_rate','annual_inc','earliest_cre_line']
    new_df=df.loc[:,df.columns.difference(target)]
    for i in range(len(target)):
        new_df['scaled_'+target[i]]=scaler_list[i].transform(df[target[i]].reshape(-1, 1))
    return new_df

In [9]:
scaled_train_set,scaler_list=scale_train_data(train_set)

/home/ubuntu/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:7: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
  import sys
/home/ubuntu/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:8: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
  
/home/ubuntu/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:10: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
  # Remove the CWD from sys.path while we load stuff.
/home/ubuntu/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:11: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
  # This is added back by InteractiveShellApp.init_path()
/home/ubuntu/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:13: FutureWarning: r

In [10]:
scaled_test_set=apply_scaler(test_set,scaler_list)
scaled_validation_set=apply_scaler(validation_set,scaler_list)

/home/ubuntu/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:6: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
  


In [11]:
scaled_test_set.head()

,binary_default,home_own_binary,inq_last_6mths,purpose_binary,term_digit,scaled_loan_amnt,scaled_int_rate,scaled_annual_inc,scaled_earliest_cre_line
11,1,2,0.0,5,3,-0.326096,-0.126828,-0.001379,-1.132109
18,1,0,0.0,8,3,-1.037033,-0.350434,0.135150,0.749444
32,1,1,1.0,5,6,-0.563075,0.619286,-0.697374,-0.145885
43,1,1,0.0,5,3,-1.226616,0.804103,-0.304777,0.839235
47,1,1,0.0,3,3,-0.563075,-0.592294,-0.729535,1.164173


In [13]:
batch_size=50
init = tf.global_variables_initializer()
start_time=time.time()
with tf.Session() as sess:
    init.run()
    
    for iteration in range(len(scaled_train_set.index.values)//batch_size):
        X=scaled_train_set.iloc[(iteration*batch_size):((iteration+1)*batch_size),1:9]
        y=scaled_train_set.iloc[(iteration*batch_size):((iteration+1)*batch_size),0]
        lo=sess.run([training_op],feed_dict={input_x:X,input_y:y})
    train_accuracy=accuracy.eval(feed_dict={input_x:X,input_y:y})
    print("train set accuracy is"+str(train_accuracy))
    #running test set and validation set
    
    test_prediction,test_accuracy=sess.run([probability,accuracy],feed_dict={input_x:scaled_test_set.iloc[:,1:9],input_y:scaled_test_set.iloc[:,0]})
    print(list(zip(scaled_test_set.iloc[:,0],test_prediction))[0:10])
    print("test set accuracy is "+str(test_accuracy))
    
    validate_prediction,validate_accuracy=sess.run([probability,accuracy],feed_dict={input_x:scaled_validation_set.iloc[:,1:9],input_y:scaled_validation_set.iloc[:,0]})
    #print(zip(validation_set.iloc[:,8],validate_prediction))
    print("validation set accuracy is "+str(validate_accuracy))
    sess.close()
    print("With Scaling, the algorithm takes --- %s seconds ---" % (time.time() - start_time))

train set accuracy is0.9
[(1, array([ 0.06530571,  0.93469423], dtype=float32)), (1, array([ 0.05084589,  0.94915414], dtype=float32)), (1, array([ 0.10884622,  0.89115381], dtype=float32)), (1, array([ 0.13246703,  0.86753291], dtype=float32)), (1, array([ 0.04542607,  0.95457399], dtype=float32)), (0, array([ 0.1732198,  0.8267802], dtype=float32)), (1, array([ 0.2075405 ,  0.79245943], dtype=float32)), (1, array([ 0.15669949,  0.84330052], dtype=float32)), (1, array([ 0.08986823,  0.91013175], dtype=float32)), (1, array([ 0.0221095 ,  0.97789055], dtype=float32))]
test set accuracy is 0.923228
validation set accuracy is 0.923206
With Scaling, the algorithm takes --- 20.388299465179443 seconds ---


# Use the scaled data to construct NN of different nodes (1 hidden unchanged)

In [14]:
import tensorflow as tf
import numpy as np

def construct_neural_layer(n_neural,input_x,name,activation_fnc=None):
    with tf.name_scope(name):
        n_inputs=int(input_x.get_shape()[1])
        stddev=2/np.sqrt(n_inputs)
        w=tf.Variable(tf.truncated_normal((n_inputs,n_neural),stddev=stddev),name="weight",dtype=tf.float32)
        b=tf.Variable(tf.zeros([n_neural]),dtype=tf.float32,name="biases")
        z=tf.add(tf.matmul(input_x,w),b)
        if activation_fnc=="sigmoid":
            return tf.sigmoid(z)
        elif activation_fnc=="relu":
            return tf.nn.relu(z)
        elif activation_fnc=="softmax":
            return tf.nn.softmax(z)
        else:
            return z
            

In [15]:
scaled_train_set.head()

,binary_default,home_own_binary,inq_last_6mths,purpose_binary,term_digit,scaled_loan_amnt,scaled_int_rate,scaled_annual_inc,scaled_earliest_cre_line
1,0,1,5.0,1,6,-1.451746,0.461849,-0.684025,0.144462
2,1,1,2.0,2,3,-1.463594,0.619286,-0.953260,0.492216
4,1,1,0.0,3,6,-1.392501,-0.126828,0.074470,-0.291979
5,1,1,3.0,4,3,-1.155522,-1.219760,-0.593005,0.895538
6,1,1,1.0,5,6,-0.918543,0.619286,-0.426076,0.984592


In [16]:
input_x=tf.placeholder(tf.float32,shape=(None,8),name="X")
input_y=tf.placeholder(tf.int32,shape=(None),name="Y")
batch_size=50
learning_rate=0.001

for n in [4,6,8,10,12,14,16]:
    hidden=construct_neural_layer(n,input_x,"Hidden_Layer",activation_fnc="relu")
    output=construct_neural_layer(2,hidden,"Output_Layer")
   
    
    loss=tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(labels=input_y,logits=output))
    accuracy=tf.reduce_mean(tf.cast(tf.nn.in_top_k(output,input_y,1),tf.float32))
    
    with tf.name_scope("train"):
        optimizer = tf.train.AdamOptimizer(learning_rate)
        training_op = optimizer.minimize(loss)
    init = tf.global_variables_initializer()
    start_time=time.time()
    with tf.Session() as sess:
        init.run()
        for iters in range(len(scaled_train_set.index.values)//batch_size):
            x=scaled_train_set.iloc[(iters*batch_size):((iters+1)*batch_size),1:9]
            y=scaled_train_set.iloc[(iters*batch_size):((iters+1)*batch_size),0]
            train_loss,train_accuracy=sess.run([training_op,accuracy],feed_dict={input_x:x,input_y:y})
            #add summary if needed later
        acc_train=accuracy.eval(feed_dict={input_x:x,input_y:y})
        acc_test=accuracy.eval(feed_dict={input_x:scaled_test_set.iloc[:,1:9],input_y:scaled_test_set.iloc[:,0]})
        acc_valid=accuracy.eval(feed_dict={input_x:scaled_validation_set.iloc[:,1:9],input_y:scaled_validation_set.iloc[:,0]})
        sess.close()
        print("with number of neural equals "+str(n)+":")
        print("Train Accuracy is "+str(acc_train))
        print("Test Accuracy is "+str(acc_test))
        print("Validation Accuracy is "+str(acc_valid))
        print("The algorithm takes --- %s seconds ---" % (time.time() - start_time))
        print()

with number of neural equals 4:
Train Accuracy is 0.9
Test Accuracy is 0.923213
Validation Accuracy is 0.923198
The algorithm takes --- 28.269792795181274 seconds ---

with number of neural equals 6:
Train Accuracy is 0.9
Test Accuracy is 0.92322
Validation Accuracy is 0.923124
The algorithm takes --- 28.60112762451172 seconds ---

with number of neural equals 8:
Train Accuracy is 0.9
Test Accuracy is 0.92319
Validation Accuracy is 0.923168
The algorithm takes --- 32.09533667564392 seconds ---

with number of neural equals 10:
Train Accuracy is 0.9
Test Accuracy is 0.92313
Validation Accuracy is 0.923011
The algorithm takes --- 30.374179363250732 seconds ---

with number of neural equals 12:
Train Accuracy is 0.9
Test Accuracy is 0.92322
Validation Accuracy is 0.923213
The algorithm takes --- 28.471309661865234 seconds ---

with number of neural equals 14:
Train Accuracy is 0.9
Test Accuracy is 0.923085
Validation Accuracy is 0.923034
The algorithm takes --- 29.1427264213562 seconds --

It seems that the number of neurals is not significantly changing the modle performance